In [23]:
#Imports
from anls_star import anls_score
import os
import ast
import json
import re
from sklearn.metrics import precision_score, recall_score, f1_score

In [24]:
#Define variables and sources
input_folder = r"E:\uni\BA\data\input\feed"
entity_folder = r"E:\uni\BA\data\input\entitiesExtract"
values_list_prompts = []
values_list_entities = []
name_files = []
banned_files = []

Now a couple of helper functions will follow.

In [25]:
def extract_values_from_entity(data):   
       # Convert the string to a dictionary
    dict_obj = json.loads(data)
    return dict_obj

In [26]:
def extract_from_prompt():
      
      for filename in os.listdir(input_folder):
            file_path = os.path.join(input_folder, filename)
        

            if os.path.isfile(file_path):
                  with open(file_path, 'r') as file:
                        text_content = file.read()

#post processing
            # Attempt to extract the dictionary-like content using regex
            match = re.search(r'\{.*\}', text_content, re.DOTALL)
            if match:
                text_content = match.group(0)
                try:
                    values = ast.literal_eval(text_content)
                except Exception as e:
                    print(f"Error in file {filename}: {e}")
                    banned_files.append(filename)   
                    values = None
            
            if values is not None:
                values["company"] = values.get("company", "").replace("\n", " ")
                values["address"] = values.get("address", "").replace(" , ", ", ")
                #delete telephone number
                text = values["address"]
                cleaned_text = re.sub(r'TEL NO\.:.*', '', text)
                values["address"] = cleaned_text
                #delete everything in brackets
                """text = values["address"]
                cleaned_text = re.sub(r'\(.*\)$', '', text)
                values["address"] = cleaned_text"""
                values_list_prompts.append(values)

<>:31: SyntaxWarning: invalid escape sequence '\('
<>:31: SyntaxWarning: invalid escape sequence '\('
C:\Users\super161\AppData\Local\Temp\ipykernel_19972\498078804.py:31: SyntaxWarning: invalid escape sequence '\('
  """text = values["address"]


In [27]:
def extract_from_entity():
    print(banned_files)
    for filename in os.listdir(entity_folder):
        if filename not in banned_files:
            file_path = os.path.join(entity_folder, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    text_content = file.read()
                
            values = extract_values_from_entity(text_content)
            #switch from SANYU STATIONERY SHOP to SANYU SUPPLY SDN BHD because not in instruction data and it is the correct owned by company...
            if values["company"] =="SANYU STATIONERY SHOP":
                values["company"] = "SANYU SUPPLY SDN BHD"
            values_list_entities.append(values)
            name_files.append(filename)
        else:
            print(f"Skipping file {filename} due to previous errors")

In [28]:
def clean_strings(strings):
    cleaned_strings = []
    for s in strings:
        # Replace ",," with ","
        if s is not None:
            s = s.replace(",,", ",")
            # Replace multiple white spaces with a single white space
            s = re.sub(r'\s+', ' ', s)
            # Append the cleaned string to the list
            cleaned_strings.append(s.strip())  # .strip() removes leading/trailing whitespace if necessary
    return cleaned_strings

The function, that orchestrates all the above helper functions:

In [29]:
def evaluation_orchestrator():
    anls_scores_all_labels = []
    anls_score_0 = []
    anls_score_1 = []
    anls_score_2 = []
    anls_score_3 = []
      
    precision = []
    recall = []
    f1 = []
      
    extract_from_prompt()
    extract_from_entity()
    for entities, prompts,name in zip(values_list_entities, values_list_prompts,name_files):
        
        entities_keys = list(entities.keys())
        prompts_keys = list(prompts.keys())
        # Extracting values from the dictionaries
        entities_values = list(entities.values())
        prompts_values = list(prompts.values())

##post processing
        prompts_values = clean_strings(prompts_values)
            
            
        # ANLS computations using only the values
        anls = anls_score(entities_values, prompts_values)
        anls_scores_all_labels.append(anls)

        key_0 = 'company'
        key_1 = 'date'
        key_2 = 'address'
        key_3 = 'total'

        #test if all keys are in the dictionary
        allowed_keys = ['company', 'date', 'address', 'total']
        if not set(prompts.keys()).issubset(allowed_keys):
            print(dictionary)
            #anls computations
        if prompts.get(key_0, '') != None:
            prompts[key_0] = prompts.get(key_0, '').upper()
        if prompts.get(key_1, '') != None:
            prompts[key_1] = prompts.get(key_1, '').upper()
        if prompts.get(key_2, '') != None:
            prompts[key_2] = prompts.get(key_2, '').upper()
        if prompts.get(key_3, '') != None: 
            prompts[key_3] = prompts.get(key_3, '').upper() 
        anls_score_0.append(anls_score(entities.get(key_0, ''), prompts.get(key_0, '')))
        anls_score_1.append(anls_score(entities.get(key_1, ''), prompts.get(key_1, '')))
        anls_score_2.append(anls_score(entities.get(key_2, ''), prompts.get(key_2, '')))
        anls_score_3.append(anls_score(entities.get(key_3, ''), prompts.get(key_3, '')))
        
        if anls < 0.96:
            print(name)
            print(entities)
            print(prompts)
            print(anls)
        
        #precision, recall, f1
        # Convert the dictionaries to sets of keys
        gt_keys = set(entities.keys())            
        ev_keys = set(prompts.keys())        
        # Union of all keys to get a comprehensive list
        all_keys = list(gt_keys.union(ev_keys))

        # Create binary labels for the presence of keys
        gt_labels = [entities.get(key) == prompts.get(key) for key in all_keys]
        ev_labels = [key in ev_keys for key in all_keys]

        # Calculate precision, recall, and F1 score
        precision.append(precision_score(gt_labels, ev_labels, zero_division=0))
        recall.append(recall_score(gt_labels, ev_labels, zero_division=0))
        f1.append(f1_score(gt_labels, ev_labels, zero_division=0))
    print(len(values_list_prompts))
    print(" anls* final average score over all labels:")
    print(round(sum(anls_scores_all_labels)/len(anls_scores_all_labels),2))
    print(" anls* final average score for company:")
    print(round(sum(anls_score_0)/len(anls_score_0),2))
    print(" anls* final average score for label date:")
    print(round(sum(anls_score_1)/len(anls_score_1),2))
    print(" anls* final average score for label address:")
    print(round(sum(anls_score_2)/len(anls_score_2),2))
    print(" anls* final average score for label total:")
    print(round(sum(anls_score_3)/len(anls_score_3),2))
    print("precision:")
    print(round(sum(precision)/len(precision),2))
    print("recall:")
    print(round(sum(recall)/len(recall),2))
    print("f1:")
    print(round(sum(f1)/len(f1),2))
evaluation_orchestrator()

Error in file X51005719902.txt: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<unknown>, line 1)
Error in file X51006389894.txt: unterminated string literal (detected at line 1) (<unknown>, line 1)
['X51005719902.txt', 'X51006389894.txt']
Skipping file X51005719902.txt due to previous errors
Skipping file X51006389894.txt due to previous errors
X00016469612.txt
{'company': 'BOOK TA .K (TAMAN DAYA) SDN BHD', 'date': '25/12/2018', 'address': 'NO.53 55,57 & 59, JALAN SAGU 18, TAMAN DAYA, 81100 JOHOR BAHRU, JOHOR.', 'total': '9.00'}
{'company': 'TAN WOON YANN BOOK TA .K(TAMAN DAYA) SDN BND', 'date': '25/12/2018', 'address': 'NO.53 55,57 & 59, JALAN SAGU 18, TAMAN DAYA, 81100 JOHOR BAHRU, JOHOR.', 'total': '9.00'}
0.9090909090909091
X00016469619.txt
{'company': 'INDAH GIFT & HOME DECO', 'date': '19/10/2018', 'address': '27, JALAN DEDAP 13, TAMAN JOHOR JAYA, 81100 JOHOR BAHRU, JOHOR.', 'total': '60.30'}
{'company': 'TAN WOON YANN INDAH GIFT